Wyznaczanie podobnych obrazów

In [ ]:
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
import plotly.graph_objects as go
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from tensorflow import random
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import json
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# Wczytaj wytrenowany model i obetnij warstwy klasyfikujące
model = load_model('../backend/best_model.keras')
feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)

# Funkcja przetwarzająca obraz
def preprocess_image(image_path, target_size=(128, 128)):
    img = load_img(image_path, target_size=target_size)
    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    return np.expand_dims(img_array, axis=0)

# Funkcja wyciągająca cechy
def extract_features(image_path):
    img_array = preprocess_image(image_path)
    features = feature_extractor.predict(img_array, verbose=0)
    return features.flatten()

# 1. Obraz zapytania (query image)
query_path = 'images/99999.jpg'
query_features = extract_features(query_path)

# 2. Ekstrakcja cech z bazy (folder z obrazami)
image_paths = df['image_path'].tolist()
base_features = np.array([extract_features(p) for p in image_paths])

# 3. Oblicz podobieństwo
similarities = cosine_similarity([query_features], base_features)[0]

# 4. Sortuj według podobieństwa
top_indices = np.argsort(similarities)[::-1]
top_matches = [(image_paths[i], similarities[i]) for i in top_indices[:5]]  # Top 5

# 5. Wyświetl najbardziej podobne obrazy
plt.figure(figsize=(12, 3))
plt.suptitle("Najbardziej podobne obrazy do query.jpg", fontsize=14)

# Pokaż obraz zapytania
plt.subplot(1, 6, 1)
plt.imshow(Image.open(query_path))
plt.axis('off')
plt.title("Zapytanie")

# Pokaż top 5 podobnych
for i, (img_path, score) in enumerate(top_matches):
    plt.subplot(1, 6, i + 2)
    plt.imshow(Image.open(img_path))
    plt.axis('off')
    plt.title(f"{score:.2f}")

plt.tight_layout()

plt.show()

for path, score in top_matches:
    print(f"{os.path.basename(path)} — similarity: {score:.4f}")